In [2]:
import os
%pwd
os.chdir("../")
%pwd

'e:\\Datasets\\Hafeez_DataSets\\chicken-Disease-Classification'

In [11]:

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

# Configuration Manager
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

CONFIG_FILE_PATH = Path("config/config.yaml")  # Ensure this is correctly defined
PARAMS_FILE_PATH = Path("params.yaml")  # Ensure this is correctly defined

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config['artifacts_root']])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config['prepare_base_model']
        create_directories([config['root_dir']])

        return PrepareBaseModelConfig(
            root_dir=Path(config['root_dir']),
            base_model_path=Path(config['base_model_path']),
            updated_base_model_path=Path(config['updated_base_model_path']),
            params_image_size=self.params['IMAGE_SIZE'],
            params_learning_rate=self.params['LEARNING_RATE'],
            params_include_top=self.params['INCLUDE_TOP'],
            params_weights=self.params['WEIGHTS'],
            params_classes=self.params['CLASSES']
        )

# Usage example:
config_manager = ConfigurationManager()
prepare_base_model_config = config_manager.get_prepare_base_model_config()
print(prepare_base_model_config)


[2024-05-22 13:54:36,137: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-22 13:54:36,143: INFO: common: yaml file: params.yaml loaded successfully]


[2024-05-22 13:54:36,428: INFO: common: created directories at: artifacts]
[2024-05-22 13:54:36,430: INFO: common: created directories at: artifacts/prepare_base_model]
PrepareBaseModelConfig(root_dir=WindowsPath('artifacts/prepare_base_model'), base_model_path=WindowsPath('artifacts/prepare_base_model/base_model.h5'), updated_base_model_path=WindowsPath('artifacts/prepare_base_model/base_model_updated.h5'), params_image_size=BoxList([224, 224, 3]), params_learning_rate=0.01, params_include_top=False, params_weights='imagenet', params_classes=2)


In [15]:
# Working on Component
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        
    
    def get_base_model(self):
        self.model = tf.keras.application.vgg16.VGG16(
            input_shape  =self.config.params_image_size,
            weights=self.config.params_weights,
            include_top = self.config.params_include_top
        )
        
        self.save_model(path=self.config.base_model_path, mode= self.model)

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0 ):
            for layer in model.layers[: freeze_till]:
                model.trainable = False
                
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units= classes,
            activation="softmax"
        )(faltten_in)
        
        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )
        
        full_model.compile(
            optimizer  =tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )
        
        full_model.summary()
        return full_model
        
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till= None,
            learning_rate=self.config.params_learning_rate
        )
        
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
        
    @staticmethod
    
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

#Creating pipeline
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model=PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e 

[2024-05-22 16:25:10,254: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-22 16:25:10,262: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-22 16:25:10,262: INFO: common: created directories at: artifacts]
[2024-05-22 16:25:10,262: INFO: common: created directories at: artifacts/prepare_base_model]


TypeError: PrepareBaseModel() takes no arguments

[2024-05-22 16:21:11,057: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-22 16:21:11,070: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-22 16:21:11,072: INFO: common: created directories at: artifacts]
[2024-05-22 16:21:11,074: INFO: common: created directories at: artifacts/prepare_base_model]


TypeError: PrepareBaseModel() takes no arguments